# RVC_CLI
Created by [Blaise](https://github.com/blaise-tk) and based on [RVC_CLI](https://github.com/blaise-tk/RVC_CLI).

- Colab inspired on [RVC v2 Disconnected](https://colab.research.google.com/drive/1XIPCP9ken63S7M6b5ui1b36Cs17sP-NS).



# Installation

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone
!git clone https://github.com/blaise-tk/RVC_CLI

In [ ]:
#@title Install
%cd RVC_CLI
!pip install -r requirements.txt

In [ ]:
#@title Download models
!python rvc/lib/tools/prerequisites_download.py

# Infer

In [ ]:
#@title Download model
#@markdown Hugging Face or Google Drive
model_link = "https://drive.google.com/file/d/1rneJ3O2IIQm151-Hci-ZKQxPhg0Xdm4o" #@param {type:"string"}

!python main.py download {model_link}

In [ ]:
#@title Run Inference
#@markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/RVC_CLI/models` for the model name (name of the folder).

import os
current_dir = os.getcwd()

model_name = "Darwin" #@param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/models/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

# List files in the model folder
files_in_folder = os.listdir(model_folder)
pth_file = next((f for f in files_in_folder if f.endswith('.pth')), None)
index_file = next((f for f in files_in_folder if f.endswith('.index')), None)

# Verificar que se encontraron ambos archivos
if pth_file is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_file)
index_file = os.path.join(model_folder, index_file)

input_path = "/content/drive/MyDrive/vocals.wav" #@param {type:"string"}
output_path = "/content/output.wav"
f0method = "rmvpe" #@param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe"] {allow-input: false}
f0up_key = 0 #@param {type:"slider", min:-24, max:24, step:0}
filter_radius = 0 #@param {type:"slider", min:0, max:10, step:0}
index_rate = 0.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 1 # @param {type:"slider", min:1, max:512, step:0}
split_audio = False #@param{type:"boolean"}
!python main.py infer {f0up_key} {filter_radius} {index_rate} {hop_length} {f0method} "{input_path}" "{output_path}" "{pth_file}" "{index_file}" {split_audio}

from IPython.display import Audio, display, clear_output
clear_output()
display(Audio(output_path))

# Train

In [ ]:
#@title Preprocess Dataset
model_name = "Darwin" #@param {type:"string"}
dataset_path = "/content/drive/MyDrive/Darwin_Dataset" #@param {type:"string"}

sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip('k'))*1000

!python main.py preprocess "{model_name}" "{dataset_path}" {sr}

In [ ]:
#@title Extract Features
model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}
f0method = "rmvpe" #@param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe"] {allow-input: false}
hop_length = 128 #@param {type:"slider", min:1, max:512, step:0}
sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip('k'))*1000

!python main.py extract {model_name} {rvc_version} {f0method} {hop_length} {sr}

In [ ]:
#@title Train
import requests
import threading
import time
import os
import shutil
import hashlib
import time

LOGS_FOLDER = '/content/RVC_CLI/logs/'
WEIGHTS_FOLDER = LOGS_FOLDER + model_name
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/RVC_Backup'

def import_google_drive_backup():
    print("Importing Google Drive backup...")
    weights_exist = False
    for root, dirs, files in os.walk(GOOGLE_DRIVE_PATH):
        for filename in files:
            filepath = os.path.join(root, filename)
            if os.path.isfile(filepath) and not filepath.startswith(os.path.join(GOOGLE_DRIVE_PATH, 'weights')):
                backup_filepath = os.path.join(LOGS_FOLDER, os.path.relpath(filepath, GOOGLE_DRIVE_PATH))
                backup_folderpath = os.path.dirname(backup_filepath)
                if not os.path.exists(backup_folderpath):
                    os.makedirs(backup_folderpath)
                    print(f'Created backup folder: {backup_folderpath}', flush=True)
                shutil.copy2(filepath, backup_filepath) # copy file with metadata
                print(f'Imported file from Google Drive backup: {filename}')
            elif filepath.startswith(os.path.join(GOOGLE_DRIVE_PATH, 'weights')) and filename.endswith('.pth'):
                weights_exist = True
                weights_filepath = os.path.join(WEIGHTS_FOLDER, os.path.relpath(filepath, os.path.join(GOOGLE_DRIVE_PATH, 'weights')))
                weights_folderpath = os.path.dirname(weights_filepath)
                if not os.path.exists(weights_folderpath):
                    os.makedirs(weights_folderpath)
                    print(f'Created weights folder: {weights_folderpath}', flush=True)
                shutil.copy2(filepath, weights_filepath) # copy file with metadata
                print(f'Imported file from weights: {filename}')
    if weights_exist:
        print("Copied weights from Google Drive backup to local weights folder.")
    else:
        print("No weights found in Google Drive backup.")
    print("Google Drive backup import completed.")

def get_md5_hash(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def copy_weights_folder_to_drive():
    destination_folder = os.path.join(GOOGLE_DRIVE_PATH, 'weights')
    try:
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        num_copied = 0
        for filename in os.listdir(WEIGHTS_FOLDER):
            if filename.endswith('.pth'):
                source_file = os.path.join(WEIGHTS_FOLDER, filename)
                destination_file = os.path.join(destination_folder, filename)
                if not os.path.exists(destination_file):
                    shutil.copy2(source_file, destination_file)
                    num_copied += 1
                    print(f"Copied {filename} to Google Drive!")

        if num_copied == 0:
            print("No new finished models found for copying.")
        else:
            print(f"Finished copying {num_copied} files to Google Drive!")

    except Exception as e:
        print(f"An error occurred while copying weights: {str(e)}")
        # You can log the error or take appropriate actions here.

def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(LOGS_FOLDER, 'last_backup_timestamps.txt')
    fully_updated = False  # boolean to track if all files are up to date

    while True:
        try:
            updated = False  # flag to check if any files were updated
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, 'r') as f:
                    last_backup_timestamps = dict(line.strip().split(':') for line in f)
            except FileNotFoundError:
                pass  # File does not exist yet, which is fine

            for root, dirs, files in os.walk(LOGS_FOLDER):
                for filename in files:
                    if filename != 'last_backup_timestamps.txt':
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                                print(f'Created backup folder: {backup_folderpath}', flush=True)
                            # check if file has changed since last backup
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if last_backup_timestamp is None or float(last_backup_timestamp) < current_timestamp:
                                shutil.copy2(filepath, backup_filepath)  # copy file with metadata
                                last_backup_timestamps[filepath] = str(current_timestamp)  # update last backup timestamp
                                if last_backup_timestamp is None:
                                    print(f'Backed up file: {filename}')
                                else:
                                    print(f'Updating backed up file: {filename}')
                                updated = True
                                fully_updated = False  # if a file is updated, all files are not up to date

            # check if any files were deleted in Colab and delete them from the backup drive
            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        print(f'Deleted file: {filepath}')
                    del last_backup_timestamps[filepath]
                    updated = True
                    fully_updated = False  # if a file is deleted, all files are not up to date

            if not updated and not fully_updated:
                print("Files are up to date.")
                fully_updated = True  # if all files are up to date, set the boolean to True
                copy_weights_folder_to_drive()
                sleep_time = 15
            else:
                sleep_time = 0.1

            with open(last_backup_timestamps_path, 'w') as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f'{filepath}:{timestamp}\n')

            time.sleep(sleep_time)  # wait for 15 seconds before checking again, or 0.1s if not fully up to date to speed up backups

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            # You can log the error or take appropriate actions here.

model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}
save_every_epoch = 10 #@param {type:"slider", min:1, max:100, step:0}
save_only_latest = False #@param{type:"boolean"}
save_every_weights = False #@param{type:"boolean"}
total_epoch = 800 #@param {type:"slider", min:1, max:10000, step:0}
sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
batch_size = 15 #@param {type:"slider", min:1, max:25, step:0}
gpu = 0 # @param {type:"number"}
sr = int(sample_rate.rstrip('k'))*1000
pitch_guidance = True #@param{type:"boolean"}
pretrained = True #@param{type:"boolean"}
custom_pretrained = False #@param{type:"boolean"}
g_pretrained_path = 'Custom Path' # @param {type:"string"}
d_pretrained_path = 'Custom Path' # @param {type:"string"}
auto_backups = True #@param{type:"boolean"}
def start_train():
  %load_ext tensorboard
  %tensorboard --logdir /content/RVC_CLI/logs/
  !python main.py train {model_name} {rvc_version} {save_every_epoch} {save_only_latest} {save_every_weights} {total_epoch} {sr} {batch_size} {gpu} {pitch_guidance} {pretrained} {custom_pretrained} {g_pretrained_path} {d_pretrained_path}

server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    backup_files()
else:
    while True:
        time.sleep(10) # sleep for 10 seconds

In [ ]:
#@title Generate index file
model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}

!python main.py index {model_name} {rvc_version}

In [ ]:
#@title Save model
#@markdown Enter the name of the model and the steps. You can find it in your `/content/RVC_CLI/logs` folder.
%cd /content
import shutil, os
model_name = "Darwin"  #@param {type:"string"}
model_epoch = 800  #@param {type:"integer"}
save_big_file = False #@param {type:"boolean"}

if os.path.exists('/content/zips'):
  shutil.rmtree('/content/zips')
print('Removed zips.')
!mkdir -p /content/zips/{model_name}/
print('Created zips.')
if f"{model_name}.pth" not in os.listdir(f'/content/RVC_CLI/weights'):
  print('There is no weight file with that name')
if not save_big_file:
  !cp /content/RVC_CLI/logs/{model_name}/added_*.index /content/zips/{model_name}/
  !cp /content/RVC_CLI/logs/{model_name}/total_*.npy /content/zips/{model_name}/
  !cp /content/RVC_CLI/weights/{model_name}.pth /content/zips/{model_name}/{model_name}{model_epoch}.pth
  %cd /content/zips
  !zip -r {model_name}.zip {model_name}
if save_big_file:
  %cd /content/RVC_CLI
  latest_steps = -1
  logs_folder = './logs/' + model_name
  for filename in os.listdir(logs_folder):#get the latest g and d files
    if filename.startswith('G_') and filename.endswith('.pth'):
      steps = int(filename.split('_')[1].split('.')[0])
      if steps > latest_steps:
        latest_steps = steps
  MODELZIP = model_name + '.zip'
  !mkdir -p /content/zips
  ZIPFILEPATH = os.path.join('/content/zips', MODELZIP)
  for filename in os.listdir(logs_folder):
    if 'G_' in filename or 'D_' in filename:
      if str(latest_steps) in filename:
        !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
    else:
      !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
  for filename in os.listdir('./weights'):
    if MODELNAME in filename:
      !zip -r {ZIPFILEPATH} {os.path.join('./weights/', filename)}

!mkdir -p /content/drive/MyDrive/RVC_Backup/
shutil.move(f'/content/zips/{model_name}.zip',f'/content/drive/MyDrive/RVC_Backup/{model_name}.zip')
%cd /content
shutil.rmtree("/content/zips")